### examines catA-v3 versus nt megablast alignments


In [59]:
from collections import defaultdict
import gzip
import subprocess
import statistics

import seaborn as sns
import pandas as pd
import matplotlib.pylab as plt

In [113]:
fh = gzip.open("catA-v3.megablast-nt-fmt6.gz")

from blast import parse

buffer = ""
def print_later(*args):
    s = " ".join(map(str,args))
    global buffer
    buffer += s + "\n"
    
def get_ref_length(ref_name):
    cmd = ["/home/ec2-user/ncbi-blast-2.10.1+/bin/blastdbcmd", "-db","/home/ec2-user/blastdb/nt","-entry",ref_name,"-outfmt","%l"]
    return int(subprocess.check_output(cmd))

df = pd.DataFrame(columns=['Assembly', 'AssemblyLen', 'Tech', 'BestRef','BestRefLen', 'LongestHit', 'QueryCov','RefCov', 'QueryDup', 'RefDup', 'IDYmean', 'IDYstdev'])

tech = dict(map(lambda x:x.split(),open("catA-v3.tech.txt").read().strip().split('\n')))

# loop over assemblies
for blast_record in parse(fh):
    #print('query id: {}'.format(blast_record.qid))
    query = blast_record.qid.decode()
    accession = query.split('.')[0]
    query_length = int(query.split("_")[3])
    ref_length = None
    best_ref = None
    query_cov_array = [0] * query_length
    ref_cov_array = None
    buffer = ""
    identities = []
    nb_alns_best_ref = 0
    longest_aln_length = 0
    
    # parse alignments between this assembly and nt
    for hit in blast_record.hits:
        for hsp in hit:
            if best_ref is None:
                # set the only reference we'll ever look at (the best one), discarding any other reference in further hsp
                best_ref = hsp.sid
                ref_length = get_ref_length(best_ref) 
                ref_cov_array = [0] * ref_length
            else:
                if hsp.sid != best_ref:
                    continue
                    
            nb_alns_best_ref += 1
            longest_aln_length = max(longest_aln_length, hsp.length)
            
            print_later('****Alignment****')
            print_later('aligns_to:', hsp.sid)
            print_later('aln_length:', hsp.length, '/', query_length)
            print_later('e value:', hsp.evalue)
            
            if hsp.pident < 90 and hsp.length > 1000:
                print("!",query,hsp.pident,hsp.length)
            
            # record which positions of the query (=assembly) are covered by an alignment to the best reference
            assert(hsp.qstart < hsp.qend)
            for i in range(hsp.qstart, hsp.qend): 
                query_cov_array[i] += 1
                
            # do same for positions of the reference
            if hsp.sstart > hsp.send:
                hsp.sstart, hsp.send =  hsp.send, hsp.sstart
            for i in range(hsp.sstart, hsp.send): 
                ref_cov_array[i] += 1
                
            identities += [hsp.pident]
            
    idy_stdev = 0 if len(identities) < 2 else statistics.stdev(identities)
    
    # determine assemblies where < 95% of the bases are covered by an alignment
    query_cov = sum([1 for x in query_cov_array if x >= 1]) / query_length
    ref_cov = sum([1 for x in ref_cov_array if x >= 1]) / ref_length
    query_dup = sum([0] + [x-1 for x in query_cov_array if x >= 2]) 
    ref_dup = sum([0] + [x-1 for x in ref_cov_array if x >= 2]) 
    if query_cov < 0.95  \
    or ref_cov   < 0.95 \
    or query_dup > 200 \
    or ref_dup > 200 \
    or longest_aln_length < query_length * 2/3:
        print("suspicious assembly (%.2f query coverage, %.2f ref coverage, %d query dup, %d ref dup, longest hit %d):" % (query_cov, ref_cov, query_dup, ref_dup, longest_aln_length) ,query)
        print(buffer)
        
        # insert into dataframe
        df.loc[-1] = [accession, query_length, tech[accession], best_ref, ref_length, longest_aln_length, query_cov, ref_cov, query_dup, ref_dup, statistics.mean(identities), idy_stdev]
        df.index = df.index + 1
        df = df.sort_index() 
    #break
   
fh.close()

suspicious assembly (0.93 query coverage, 1.00 ref coverage, 0 query dup, 0 ref dup, longest hit 30094): SRR1191915.coronaspades.NODE_1_length_32532_cluster_1_candidate_1_domains_36
****Alignment****
aligns_to: JX869059.2
aln_length: 30094 / 32532
e value: 0.0

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 8 query dup, 1206 ref dup, longest hit 29911): SRR11140749.coronaspades.NODE_1_length_31251_cluster_1_candidate_1_domains_38
****Alignment****
aligns_to: MT121215.1
aln_length: 29911 / 31251
e value: 0.0
****Alignment****
aligns_to: MT121215.1
aln_length: 480 / 31251
e value: 0.0
****Alignment****
aligns_to: MT121215.1
aln_length: 272 / 31251
e value: 5.71e-123
****Alignment****
aligns_to: MT121215.1
aln_length: 252 / 31251
e value: 3.49e-110
****Alignment****
aligns_to: MT121215.1
aln_length: 227 / 31251
e value: 7.6e-102

suspicious assembly (1.00 query coverage, 0.84 ref coverage, 0 query dup, 0 ref dup, longest hit 25210): SRR11494506.coronaspades.NODE_1_length_252

suspicious assembly (1.00 query coverage, 0.94 ref coverage, 0 query dup, 0 ref dup, longest hit 27986): SRR11578301.coronaspades.NODE_1_length_27986_cluster_1_candidate_1_domains_36
****Alignment****
aligns_to: MT628262.1
aln_length: 27986 / 27986
e value: 0.0

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 8 query dup, 163 ref dup, longest hit 15955): SRR11621817.coronaspades.NODE_1_length_29932_cluster_1_candidate_1_domains_39
****Alignment****
aligns_to: MT627434.1
aln_length: 15955 / 29932
e value: 0.0
****Alignment****
aligns_to: MT627434.1
aln_length: 13797 / 29932
e value: 0.0
****Alignment****
aligns_to: MT627434.1
aln_length: 109 / 29932
e value: 1.02e-45
****Alignment****
aligns_to: MT627434.1
aln_length: 34 / 29932
e value: 0.000501
****Alignment****
aligns_to: MT627434.1
aln_length: 31 / 29932
e value: 0.023

suspicious assembly (1.00 query coverage, 0.90 ref coverage, 0 query dup, 0 ref dup, longest hit 25212): SRR10829954.coronaspades.NODE_1_length_25269_cl

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 63 query dup, 201 ref dup, longest hit 27986): SRR6713902.coronaspades.NODE_1_length_28120_cluster_1_candidate_1_domains_38
****Alignment****
aligns_to: KR265814.1
aln_length: 27986 / 28120
e value: 0.0
****Alignment****
aligns_to: KR265814.1
aln_length: 131 / 28120
e value: 1.65e-23
****Alignment****
aligns_to: KR265814.1
aln_length: 72 / 28120
e value: 1.65e-23

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 6 query dup, 330 ref dup, longest hit 28021): ERR3013343.coronaspades.NODE_1_length_28355_cluster_1_candidate_1_domains_37
****Alignment****
aligns_to: LT898439.1
aln_length: 28021 / 28355
e value: 0.0
****Alignment****
aligns_to: LT898439.1
aln_length: 127 / 28355
e value: 9.51e-56
****Alignment****
aligns_to: LT898439.1
aln_length: 136 / 28355
e value: 4.42e-54
****Alignment****
aligns_to: LT898439.1
aln_length: 70 / 28355
e value: 9.99e-21

suspicious assembly (0.91 query coverage, 1.00 ref coverage, 0 

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 62 query dup, 207 ref dup, longest hit 27988): SRR6713767.coronaspades.NODE_1_length_28129_cluster_1_candidate_1_domains_40
****Alignment****
aligns_to: KF650373.1
aln_length: 27988 / 28129
e value: 0.0
****Alignment****
aligns_to: KF650373.1
aln_length: 137 / 28129
e value: 7.61e-27
****Alignment****
aligns_to: KF650373.1
aln_length: 72 / 28129
e value: 3.54e-25

suspicious assembly (1.00 query coverage, 0.88 ref coverage, 0 query dup, 0 ref dup, longest hit 26412): ERR4181768.coronaspades.NODE_1_length_26418_cluster_1_candidate_1_domains_34
****Alignment****
aligns_to: MT325599.1
aln_length: 26412 / 26418
e value: 0.0

suspicious assembly (0.99 query coverage, 1.00 ref coverage, 0 query dup, 207 ref dup, longest hit 30995): SRR9430938.coronaspades.NODE_1_length_31509_cluster_1_candidate_1_domains_33
****Alignment****
aligns_to: FJ425188.1
aln_length: 30995 / 31509
e value: 0.0
****Alignment****
aligns_to: FJ425188.1
aln_len

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 0 query dup, 1165 ref dup, longest hit 22387): SRR11772243.coronaspades.NODE_1_length_30990_cluster_1_candidate_1_domains_38
****Alignment****
aligns_to: MT450890.1
aln_length: 22387 / 30990
e value: 0.0
****Alignment****
aligns_to: MT450890.1
aln_length: 8600 / 30990
e value: 0.0

suspicious assembly (0.92 query coverage, 1.00 ref coverage, 0 query dup, 47 ref dup, longest hit 29713): SRR1030058.coronaspades.NODE_1_length_32413_cluster_1_candidate_1_domains_47
****Alignment****
aligns_to: FJ882957.1
aln_length: 29713 / 32413
e value: 0.0
****Alignment****
aligns_to: FJ882957.1
aln_length: 48 / 32413
e value: 8.96e-12

suspicious assembly (1.00 query coverage, 1.00 ref coverage, 0 query dup, 12 ref dup, longest hit 19372): SRR11771957.coronaspades.NODE_1_length_29877_cluster_1_candidate_1_domains_46
****Alignment****
aligns_to: MT482127.1
aln_length: 19372 / 29877
e value: 0.0
****Alignment****
aligns_to: MT482127.1
aln_lengt

suspicious assembly (0.95 query coverage, 1.00 ref coverage, 0 query dup, 0 ref dup, longest hit 30097): SRR1193924.coronaspades.NODE_1_length_31708_cluster_1_candidate_1_domains_36
****Alignment****
aligns_to: JX869059.2
aln_length: 30097 / 31708
e value: 0.0

suspicious assembly (1.00 query coverage, 0.94 ref coverage, 113 query dup, 0 ref dup, longest hit 28016): SRR11777734.coronaspades.NODE_1_length_28026_cluster_1_candidate_1_domains_39
****Alignment****
aligns_to: MT079851.1
aln_length: 28016 / 28026
e value: 0.0
****Alignment****
aligns_to: MT079851.1
aln_length: 115 / 28026
e value: 7.37e-47

suspicious assembly (0.91 query coverage, 1.00 ref coverage, 0 query dup, 0 ref dup, longest hit 30098): SRR1191668.coronaspades.NODE_1_length_33011_cluster_1_candidate_1_domains_36
****Alignment****
aligns_to: JX869059.2
aln_length: 30098 / 33011
e value: 0.0

suspicious assembly (1.00 query coverage, 0.92 ref coverage, 0 query dup, 0 ref dup, longest hit 25054): SRR5872115.coronaspades.

In [108]:
df.to_csv("catA-v3.blastn-QC.txt")
df
print(df.to_markdown())
#df[40:80]

|     | Assembly    |   AssemblyLen | Tech            | BestRef    |   BestRefLen |   LongestHit |   QueryCov |    RefCov |   QueryDup |   RefDup |   IDYmean |   IDYstdev |
|----:|:------------|--------------:|:----------------|:-----------|-------------:|-------------:|-----------:|----------:|-----------:|---------:|----------:|-----------:|
|   0 | ERR1301465  |         31358 | ILLUMINA        | MH687969.1 |        31352 |        31234 |   0.998246 | 0.998979  |        394 |      357 |   98.4641 | 4.33473    |
|   1 | SRR11140744 |         30087 | ILLUMINA        | MT039887.1 |        29879 |        29867 |   0.999867 | 0.999565  |          5 |      222 |  100      | 0          |
|   2 | SRR2146117  |         26537 | ILLUMINA        | AY597011.2 |        29926 |        26537 |   0.999962 | 0.899519  |       1508 |     1125 |   94.4134 | 2.47587    |
|   3 | SRR11954068 |         28303 | ILLUMINA        | MT520382.1 |        29866 |        28303 |   0.999965 | 0.947633  |          0 